# Calcul du taux d'émission pour les véhicules

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns

In [17]:
essence=pd.read_csv("emissions_essence.csv")
diesel=pd.read_csv("emissions_diesel.csv")
euro=pd.read_csv("norme_euro.csv")
data=pd.read_csv("Donnees-sur-le-parc-de-vehicule-au-niveau-regional.2023-05.csv", sep=';',header=[1])
data = data.loc[data["CATEGORIE_VEHICULE"] == "Véhicule particulier"]
data.reset_index(drop=True, inplace=True)

In [18]:
essence

,Norme,Euro 1,Euro 2,Euro 3,Euro 4,Euro 5[25],Euro 6b[26],Euro 6c,Euro 6d -TEMP,Euro 6d
0,Oxydes d'azote (NOx),-,-,150,80,60,60,60,60,60
1,Monoxyde de carbone (CO),2 720,2 200,2 200,1 000,1 000,1 000,1 000,1 000,1 000
2,Hydrocarbures (HC),-,-,200,100,100,100,100,100,100
3,Hydrocarbures non méthaniques (HCNM),-,-,-,-,68,68,68,68,68
4,HC + NOX,970,500,-,-,-,-,-,-,-
5,Particules (PM),-,-,-,-,5[* 1],5[* 1],45,45,45
6,Particules (PN) (nb/km),-,-,-,-,-,6 × 1012[* 2],6 × 1011,6 × 1011,6 × 1011


In [19]:
diesel

,Norme,Euro 1,Euro 2,Euro 3,Euro 4,Euro 5a,Euro 5b,Euro 6b,Euro 6c,Euro 6d -TEMP,Euro 6d
0,Oxydes d'azote (NOx),-,-,500,250,180,180,80,80,80,80
1,Monoxyde de carbone (CO),2 720,1 000,640,500,500,500,500,500,500,500
2,Hydrocarbures (THC),-,-,-,-,-,-,-,-,-,-
3,Hydrocarbures non méthaniques (HCNM),-,-,-,-,-,-,-,-,-,-
4,HC + NOX,970,700,560,300,230,230,170,170,170,170
5,Particules (PM),140,80,50,25,5,45,45,45,45,45
6,Particules (PN) (nb/km),-,-,-,-,6 × 1011,6 × 1011,6 × 1011,6 × 1011,6 × 1011,6 × 1011


In [20]:
euro

,Norme,Mise en service des véhicules
0,Euro 1,1er janvier 1993
1,Euro 2 (voitures),1er janvier 1997
2,Euro 2 (Véhicules utilitaires légers),1er octobre 1997
3,Euro 3,1er janvier 2001
4,Euro 4,1er janvier 2006
5,Euro 5a,1er janvier 2011
6,Euro 5b,1er janvier 2013
7,Euro 6b,1er septembre 2015
8,Euro 6c,1er septembre 2018
9,Euro 6d-TEMP[8],1er septembre 2019


In [21]:
data.head()

,REGION_CODE,REGION_LIBELLE,CLASSE_VEHICULE,CATEGORIE_VEHICULE,CARBURANT,AGE,STATUT_UTILISATEUR,CRITAIR,PARC_2011,PARC_2012,PARC_2013,PARC_2014,PARC_2015,PARC_2016,PARC_2017,PARC_2018,PARC_2019,PARC_2020,PARC_2021,PARC_2022
0,84,Auvergne-Rhône-Alpes,vp,Véhicule particulier,Diesel,0 an,Professionnel,Crit'Air 1,0,2,1,3,0,5,0,0,0,0,0,0
1,84,Auvergne-Rhône-Alpes,vp,Véhicule particulier,Diesel,0 an,Professionnel,Crit'Air 2,22840,60245,58627,56823,55903,55948,59274,62531,56444,53142,37552,29362
2,84,Auvergne-Rhône-Alpes,vp,Véhicule particulier,Diesel,0 an,Professionnel,Crit'Air 3,32626,2456,54,1,1,1,1,0,0,0,0,0
3,84,Auvergne-Rhône-Alpes,vp,Véhicule particulier,Diesel,0 an,Professionnel,Crit'Air 4,7,1,0,0,1,0,0,0,0,0,0,0
4,84,Auvergne-Rhône-Alpes,vp,Véhicule particulier,Diesel,0 an,Professionnel,Crit'Air 5,0,1,0,0,0,0,0,0,0,0,0,0


Il sera facile de donner la norme pour tout les autres véhicules entrant à partir de 2021 puisqu'elle sera "euro 6d". Je vais ici 
donner le nombre de véhicule pour chaque norme de 2011 à 2020.

In [50]:
variables = ['PARC_2011','PARC_2012','PARC_2013','PARC_2014','PARC_2015','PARC_2016','PARC_2017','PARC_2018','PARC_2019','PARC_2020','PARC_2021','PARC_2022']
age=data['AGE']
data.groupby('AGE')[variables].sum()

,PARC_2011,PARC_2012,PARC_2013,PARC_2014,PARC_2015,PARC_2016,PARC_2017,PARC_2018,PARC_2019,PARC_2020,PARC_2021,PARC_2022
AGE,,,,,,,,,,,,
0 an,2258392,2200521,1908256,1816376,1829078,1946394,2041820,2128627,2212355,2256167,1715340,1731409
1 an,2361172,2318370,2282950,1994836,1897864,1887806,1998855,2104174,2205420,2292669,2367312,1799201
10 ans,1888623,1947483,1870484,1814733,1852111,1913669,1900201,1995713,1947277,2090397,2066119,2033963
11 ans,1815529,1798744,1875905,1814768,1771707,1806323,1860400,1847016,1934246,1901579,2047537,2019865
12 ans,1531300,1690943,1706460,1791394,1746934,1704957,1732797,1777400,1755545,1857357,1834818,1974747
13 ans,1243233,1413170,1592026,1622746,1713673,1673520,1627854,1648061,1663697,1675421,1783118,1764566
14 ans,1375653,1114223,1296215,1484330,1529647,1613612,1573964,1526234,1507121,1526198,1577662,1684208
15 ans,1152522,1219972,1006689,1191825,1386977,1433527,1504495,1467209,1388701,1369336,1417799,1484766
16 ans,1038152,993729,1073909,898822,1085032,1273563,1315678,1375340,1301885,1224991,1242182,1302719


In [51]:
age
    

0                  0 an
1                  0 an
2                  0 an
3                  0 an
4                  0 an
              ...      
14255            24 ans
14256            24 ans
14257    25 ans et plus
14258    25 ans et plus
14259    25 ans et plus
Name: AGE, Length: 14260, dtype: object

In [52]:
ages=age.unique()

In [53]:
ages_chiffres = [int(re.search(r'\d+', age).group()) for age in ages]

print(ages_chiffres)

NameError: name 're' is not defined